In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1709499088405_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ClimateSparkApp").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_emissions = spark.read.option("header", "true").csv("s3://spulido1lab1/co2_emissions/co2_emission.csv")
df_temperature = spark.read.option("header", "true").csv("s3://spulido1lab1/land-temperature-by-country/GlobalLandTemperaturesByCountry.csv")

df_emissions.show()
df_temperature.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+----+----------------+
|     entity|code|year|annual_emissions|
+-----------+----+----+----------------+
|Afghanistan| AFG|1949|           14656|
|Afghanistan| AFG|1950|           84272|
|Afghanistan| AFG|1951|           91600|
|Afghanistan| AFG|1952|           91600|
|Afghanistan| AFG|1953|          106256|
|Afghanistan| AFG|1954|          106256|
|Afghanistan| AFG|1955|          153888|
|Afghanistan| AFG|1956|          183200|
|Afghanistan| AFG|1957|          293120|
|Afghanistan| AFG|1958|          329760|
|Afghanistan| AFG|1959|       384571.42|
|Afghanistan| AFG|1960|       413883.42|
|Afghanistan| AFG|1961|        490797.7|
|Afghanistan| AFG|1962|       688594.27|
|Afghanistan| AFG|1963|       706735.98|
|Afghanistan| AFG|1964|       838550.83|
|Afghanistan| AFG|1965|      1006916.53|
|Afghanistan| AFG|1966|      1091158.82|
|Afghanistan| AFG|1967|      1281865.11|
|Afghanistan| AFG|1968|      1223389.69|
+-----------+----+----+----------------+
only showing top

In [12]:
from pyspark.sql import functions as F

top_years_emissions = df_emissions.groupBy("year") \
                        .agg(F.sum("annual_emissions") \
                        .alias("total_emissions")) \
                        .orderBy(F.desc("total_emissions")) \
                        .limit(10)


top_years_emissions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+
|year|     total_emissions|
+----+--------------------+
|2017|   8.947275082983E10|
|2016|8.824961753229005E10|
|2015|8.768654789835999E10|
|2014|8.762096132615002E10|
|2013|8.700744654957999E10|
|2012|8.665079532765002E10|
|2011|8.524360993775002E10|
|2010|8.238109186037001E10|
|2008|7.990947993998001E10|
|2009|7.867301490757996E10|
+----+--------------------+

In [14]:
hottest_countries_all_years = df_temperature.groupBy("Country") \
    .agg(F.avg("AverageTemperature").alias("average_temperature")) \
    .orderBy(F.desc("average_temperature")).limit(10)


hottest_countries_all_years.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|             Country|average_temperature|
+--------------------+-------------------+
|            Djibouti| 28.816602533172542|
|                Mali| 28.441976570819996|
|        Burkina Faso| 28.083506760410998|
|             Senegal|  27.96737499999997|
|               Aruba|  27.92039022051774|
|United Arab Emirates| 27.693994700582884|
|          Mauritania| 27.620255504352293|
|              Gambia| 27.538551816239316|
|               Niger| 27.458972540045757|
|             Curaçao| 27.353414669223397|
+--------------------+-------------------+

In [15]:
hottest_years = df_temperature.withColumn("year", F.year(F.to_date("dt", "yyyy-MM-dd"))) \
    .groupBy("year") \
    .agg(F.avg("AverageTemperature").alias("average_temperature")) \
    .orderBy(F.desc("average_temperature")) \
    .limit(10)


hottest_years.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------------+
|year|average_temperature|
+----+-------------------+
|2013| 19.877007153806858|
|2010| 19.629806129476588|
|2007|  19.61039428374656|
|1998| 19.596783057851248|
|2002| 19.565523415977967|
|2009|  19.54638533057848|
|2006|   19.5391349862259|
|2003|   19.5250867768595|
|2005| 19.499590220385706|
|2012|   19.4687441460055|
+----+-------------------+

In [18]:
countries_with_highest_avg_emissions = df_emissions.groupBy("entity") \
    .agg(F.avg("annual_emissions").alias("average_annual_emissions")) \
    .orderBy(F.desc("average_annual_emissions")).limit(10)


countries_with_highest_avg_emissions.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+
|              entity|average_annual_emissions|
+--------------------+------------------------+
|               World|     5.899791376895131E9|
|              Russia|    1.7049000668898306E9|
|       United States|    1.4957990314644194E9|
|               EU-28|     1.321566501059925E9|
|               China|     7.879389530948819E8|
|Asia and Pacific ...|     6.668811690486891E8|
|      Europe (other)|     5.900381650936329E8|
|             Ukraine|    4.3722688639152545E8|
|               Japan|    4.1536405464533335E8|
|             Germany|    4.0073287623141575E8|
+--------------------+------------------------+

In [19]:
avg_emissions = df_emissions.groupBy("entity") \
    .agg(F.avg("annual_emissions").alias("average_annual_emissions"))


avg_temperature = df_temperature.groupBy("Country") \
    .agg(F.avg("AverageTemperature").alias("average_temperature"))


countries_with_highest_avg_emissions_and_temp = avg_emissions \
    .join(avg_temperature, avg_emissions.entity == avg_temperature.Country) \
    .orderBy(F.desc("average_annual_emissions")) \
    .limit(10)

countries_with_highest_avg_emissions_and_temp.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------------+--------------+-------------------+
|        entity|average_annual_emissions|       Country|average_temperature|
+--------------+------------------------+--------------+-------------------+
|        Russia|    1.7049000668898306E9|        Russia| -5.521881912681911|
| United States|    1.4957990314644194E9| United States|  8.603572522159553|
|         China|     7.879389530948819E8|         China|  6.612731031349375|
|       Ukraine|    4.3722688639152545E8|       Ukraine|  7.653507264687312|
|         Japan|    4.1536405464533335E8|         Japan|  11.95190733590733|
|       Germany|    4.0073287623141575E8|       Germany|  8.152496209728362|
|United Kingdom|    2.8865563913258433E8|United Kingdom|  8.489325331648798|
|   South Korea|    2.1625504422643837E8|   South Korea| 11.849048745173732|
|    Kazakhstan|    2.0848503211864406E8|    Kazakhstan|  5.202144758735442|
|         India|    1.8967915343945312E8|         India|  23.87378947368426|